<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/ParticleSimulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [233]:
import numpy as np
import time
import math
from IPython.display import clear_output

class Particle:
    def __init__(self, x, y, vx, vy, index=0):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.index = index

    def move(self):
        self.x += self.vx
        self.y += self.vy

class Simulator:
    def __init__(self, width, height, collision_radius=0.8):
        self.width = width
        self.height = height
        self.collision_radius = collision_radius
        self.square_radius = self.collision_radius**2
        self.particles = []

    def add_particle(self, x, y, vx, vy):
        self.particles.append(Particle(x, y, vx, vy, len(self.particles)))

    def simulate_step(self):

        for p in self.particles:
            p.move()

        for i, p1 in enumerate(self.particles):
            for p2 in self.particles[i+1:]:
                dx = p1.x - p2.x
                dy = p1.y - p2.y
                if dx*dx + dy*dy < self.square_radius:
                    p1.vx, p2.vx = p2.vx, p1.vx
                    p1.vy, p2.vy = p2.vy, p1.vy
                    return True
        return False

class Visualizer:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def create_matrix(self, particles):
        matrix = np.zeros((self.height, self.width), dtype=int)
        for p in particles:
            x, y = int(round(p.x)), int(round(p.y))
            if 0 <= x < self.width and 0 <= y < self.height:
                matrix[y, x] = p.index + 1
        return matrix

    def print_matrix(self, matrix):
        for row in matrix:
            print(' '.join(map(str, row)))

    def print_particles_info(self, particles):
        for p in particles:
            print(f"Частица {p.index}: ({p.x:.1f}, {p.y:.1f}) v=({p.vx:.1f}, {p.vy:.1f})")

def main():
    width, height = 40, 3
    collision_radius = 3

    sim = Simulator(width, height, collision_radius)
    vis = Visualizer(width, height)

    sim.add_particle(0, 1, 1, 0)
    sim.add_particle(10, 1, 0.1, 0)
    sim.add_particle(20, 1, -1, 0)
    sim.add_particle(30, 1, -1, 0)

    for step in range(30):
        clear_output(wait=True)
        matrix = vis.create_matrix(sim.particles)
        vis.print_matrix(matrix)
        vis.print_particles_info(sim.particles)

        sim.simulate_step()
        time.sleep(0.1)

if __name__ == "__main__":
    main()

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
Частица 0: (-13.0, 1.0) v=(-1.0, 0.0)
Частица 1: (-2.7, 1.0) v=(-1.0, 0.0)
Частица 2: (14.6, 1.0) v=(0.1, 0.0)
Частица 3: (35.0, 1.0) v=(1.0, 0.0)


In [229]:
import numpy as np
import time
import random
from IPython.display import clear_output

class Particle:
    def __init__(self, x, y, vx, vy, index=0):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.index = index

    def move(self):
        self.x += self.vx
        self.y += self.vy

    def add_random_velocity(self, max_change):
        self.vx += random.uniform(-max_change, max_change)
        self.vy += random.uniform(-max_change, max_change)

class Simulator:
    def __init__(self, width, height, collision_radius=0.8, velocity_change=0.001):
        self.width = width
        self.height = height
        self.collision_radius = collision_radius
        self.velocity_change = velocity_change
        self.square_radius = self.collision_radius**2
        self.particles = []

    def add_particle(self, x, y, vx, vy):
        self.particles.append(Particle(x, y, vx, vy, len(self.particles)))

    def simulate_step(self):
        for p in self.particles:
            p.move()

        for i, p1 in enumerate(self.particles):
            for p2 in self.particles[i+1:]:
                dx = p1.x - p2.x
                dy = p1.y - p2.y
                if dx*dx + dy*dy < self.square_radius:
                    p1.vx, p2.vx = p2.vx, p1.vx
                    p1.vy, p2.vy = p2.vy, p1.vy

        for p in self.particles:
            p.add_random_velocity(self.velocity_change)

class Visualizer:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def create_matrix(self, particles):
        matrix = np.zeros((self.height, self.width), dtype=int)
        for p in particles:
            x, y = int(round(p.x)), int(round(p.y))
            if 0 <= x < self.width and 0 <= y < self.height:
                matrix[y, x] = p.index + 1
        return matrix

    def print_matrix(self, matrix):
        for row in matrix:
            print(' '.join(map(str, row)))

    def print_particles_info(self, particles):
        for p in particles:
            print(f"Частица {p.index}: ({p.x:.1f}, {p.y:.1f}) v=({p.vx:.1f}, {p.vy:.1f})")

def main():
    width, height = 40, 10
    collision_radius = 1
    velocity_change = 0.005

    sim = Simulator(width, height, collision_radius, velocity_change)
    vis = Visualizer(width, height)

    for i in range(10, 20, 3):
        sim.add_particle(i, height//2, 0, 0)

    sim.add_particle(30, height//2, -1, 0)

    for step in range(30):
        clear_output(wait=True)
        matrix = vis.create_matrix(sim.particles)
        vis.print_matrix(matrix)
        vis.print_particles_info(sim.particles)

        collision = sim.simulate_step()

        time.sleep(0.1)

if __name__ == "__main__":
    main()

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 2 0 0 3 0 0 0 0 4 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
Частица 0: (-1.2, 4.7) v=(-1.0, -0.0)
Частица 1: (10.0, 5.0) v=(0.0, 0.0)
Частица 2: (13.0, 5.2) v=(-0.0, 0.0)
Частица 3: (17.6, 4.8) v=(0.0, -0.0)
Частица 4: (20.2, 4.6) v=(0.0, -0.0)


In [353]:
import numpy as np
import time
import random
from IPython.display import clear_output

class Particle:
    def __init__(self, x, y, vx, vy, mass=1.0, index=0):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.mass = mass
        self.index = index

    def move(self):
        self.x += self.vx
        self.y += self.vy

    def add_random_velocity(self, max_change):
        self.vx += random.uniform(-max_change, max_change)
        self.vy += random.uniform(-max_change, max_change)

class Simulator:
    def __init__(self, width, height, collision_radius=0.8, velocity_change=0.001):
        self.width = width
        self.height = height
        self.collision_radius = collision_radius
        self.velocity_change = velocity_change
        self.square_radius = self.collision_radius**2
        self.particles = []

    def add_particle(self, x, y, vx, vy, mass=1.0):
        self.particles.append(Particle(x, y, vx, vy, mass, len(self.particles)))

    def simulate_step(self):
        for p in self.particles:
            p.move()

        for i, p1 in enumerate(self.particles):
            for p2 in self.particles[i+1:]:
                dx = p1.x - p2.x
                dy = p1.y - p2.y
                if dx*dx + dy*dy < self.square_radius:
                    m1, m2 = p1.mass, p2.mass
                    total_mass = m1 + m2

                    new_vx1 = (p1.vx * (m1 - m2) + 2 * m2 * p2.vx) / total_mass
                    new_vy1 = (p1.vy * (m1 - m2) + 2 * m2 * p2.vy) / total_mass
                    new_vx2 = (p2.vx * (m2 - m1) + 2 * m1 * p1.vx) / total_mass
                    new_vy2 = (p2.vy * (m2 - m1) + 2 * m1 * p1.vy) / total_mass

                    p1.vx, p1.vy = new_vx1, new_vy1
                    p2.vx, p2.vy = new_vx2, new_vy2

        for p in self.particles:
            p.add_random_velocity(self.velocity_change)

class Visualizer:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def create_matrix(self, particles):
        matrix = np.zeros((self.height, self.width), dtype=int)
        for p in particles:
            x, y = int(round(p.x)), int(round(p.y))
            if 0 <= x < self.width and 0 <= y < self.height:
                matrix[y, x] = p.index + 1
        return matrix

    def print_matrix(self, matrix):
        for row in matrix:
            for cell in row:
                if cell > 0:
                    print('●', end=' ')
                else:
                    print('·', end=' ')
            print()

    def print_enhanced_matrix(self, particles):
        display_matrix = [['·' for _ in range(self.width)] for _ in range(self.height)]

        for p in particles:
            x, y = int(round(p.x)), int(round(p.y))
            if 0 <= x < self.width and 0 <= y < self.height:
                display_matrix[y][x] = '●'

        for row in display_matrix:
            print(' '.join(row))

    def print_particles_info(self, particles):
        for p in particles:
            print(f"Частица {p.index}: ({p.x:.1f}, {p.y:.1f}) v=({p.vx:.1f}, {p.vy:.1f}) m={p.mass:.1f}")

def main():
    width, height = 40, 30
    collision_radius = 1
    velocity_change = 0.001

    sim = Simulator(width, height, collision_radius, velocity_change)
    vis = Visualizer(width, height)

    for i in range(10, 20):
        for j in range(5, 25):
            mass = random.uniform(1, 50)
            sim.add_particle(i, j, 0, 0, mass)

    sim.add_particle(30, height//2, -1, 0, mass=1000.0)

    for step in range(30):
        clear_output(wait=True)
        vis.print_enhanced_matrix(sim.particles)
        # vis.print_particles_info(sim.particles)
        collision = sim.simulate_step()

        time.sleep(0.05)

if __name__ == "__main__":
    main()

· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·
· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·
· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·
· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·
· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·
· · · · · · · · · · ● ● ● ● ● ● ● ● ● ● · · · · · · · · · · · · · · · · · · · ·
· · · · · · · · · · ● ● ● ● ● ● ● ● ● ● · · · · · · · · · · · · · · · · · · · ·
· · · · · · · · · · ● ● ● ● ● ● ● ● ● ● · · · · · · · · · · · · · · · · · · · ·
· · · · · · · · · · ● ● ● ● ● ● ● ● ● ● · · · · · · · · · · · · · · · · · · · ·
· · · · · · · · · · ● ● ● ● ● ● ● ● ● ● · · · · · · · · · · · · · · · · · · · ·
· · · · · · · · · · ● ● ● ● ● ● ● ● ● ● · · · · · · · · · · · · · · · · · · · ·
· · · · · · · · · · ● ● ● ● ● ● ● ● ● ● · · · · · · · · · · · · · · · · · · · ·
· · · · · · · · · ● · ● ● ● ● ● ● ● ● · 